In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/synth2023/processed/patient_split.json
/kaggle/input/synth2023/processed/CT_slices/1BB205_axial_slice_111.npy
/kaggle/input/synth2023/processed/CT_slices/1BA201_axial_slice_128.npy
/kaggle/input/synth2023/processed/CT_slices/1BC014_axial_slice_086.npy
/kaggle/input/synth2023/processed/CT_slices/1BA001_axial_slice_125.npy
/kaggle/input/synth2023/processed/CT_slices/1BC041_axial_slice_074.npy
/kaggle/input/synth2023/processed/CT_slices/1BB041_axial_slice_072.npy
/kaggle/input/synth2023/processed/CT_slices/1BB007_axial_slice_078.npy
/kaggle/input/synth2023/processed/CT_slices/1BC083_axial_slice_085.npy
/kaggle/input/synth2023/processed/CT_slices/1BC064_axial_slice_091.npy
/kaggle/input/synth2023/processed/CT_slices/1BB051_axial_slice_053.npy
/kaggle/input/synth2023/processed/CT_slices/1BB011_axial_slice_091.npy
/kaggle/input/synth2023/processed/CT_slices/1BA185_axial_slice_094.npy
/kaggle/input/synth2023/processed/CT_slices/1BB062_axial_slice_119.npy
/kaggle/input/synth2023/

In [2]:
# Full training script — paste into a Colab cell and run.
# Requirements:
# pip install pytorch-msssim
# (ms-ssim used optionally below; if not installed you can remove that import/usage)

import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.utils as vutils
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio, structural_similarity, mean_squared_error
from google.colab import drive
from torch.amp import autocast, GradScaler
import matplotlib.pyplot as plt
import pandas as pd
import random

# mount drive
#drive.mount('/content/drive', force_remount=False)

# -------------------------
# Config
# -------------------------
class Config:
    data_root = "/kaggle/input/synth2023"
    batch_size = 32
    num_epochs = 80
    lr_G = 2e-4
    lr_D = 1e-4
    beta1 = 0.5
    image_size = 256
    save_dir = "/kaggle/working/Model_InstNorm_FM"
    validate_every = 5
    sample_grid_n = 4
    max_val_batches = 50
    lambda_L1_start = 100.0    # starting L1 weight (we will decay toward 10)
    lambda_fm = 10.0          # feature-matching weight
    noise_std = 0.03          # gaussian noise std for D inputs
    label_real = 0.9
    label_fake = 0.1
cfg = Config()

cfg = Config()

os.makedirs(cfg.save_dir, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
torch.backends.cudnn.benchmark = True

# reproducibility (optional)
seed = 42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def denormalize(tensor, hu_min=-1000.0, hu_max=2000.0):
    # tensor in [-1,1] → HU space
    return 0.5 * (tensor + 1.0) * (hu_max - hu_min) + hu_min

# -------------------------
# Cached Dataset (preload into RAM)
# -------------------------
class MRICtDataset(Dataset):
    def __init__(self, split, split_json, mri_dir, ct_dir, image_size=256, augment=False):
        with open(split_json, "r") as f:
            splits = json.load(f)
        if "files" not in splits or split not in splits["files"]:
            raise ValueError("Invalid split JSON or split name")
        self.files = splits["files"][split]
        self.mri_slices = []
        self.ct_slices = []
        self.augment = augment

        print(f"Preloading {len(self.files)} slices into memory for '{split}'...")
        for fname in tqdm(self.files):
            mri = np.load(os.path.join(mri_dir, fname)).astype(np.float32)
            ct  = np.load(os.path.join(ct_dir,  fname)).astype(np.float32)
            if mri.ndim == 2: mri = np.expand_dims(mri, 0)
            if ct.ndim == 2:  ct  = np.expand_dims(ct, 0)
            # ensure shape (1,H,W)
            self.mri_slices.append(torch.from_numpy(mri))
            self.ct_slices.append(torch.from_numpy(ct))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        mri = self.mri_slices[idx].clone()
        ct  = self.ct_slices[idx].clone()

        # simple paired augmentations (safe for aligned pairs)
        if self.augment:
            if torch.rand(1) < 0.5:
                mri = torch.flip(mri, dims=[2]); ct = torch.flip(ct, dims=[2])
            if torch.rand(1) < 0.5:
                mri = torch.flip(mri, dims=[1]); ct = torch.flip(ct, dims=[1])
            # random 90-degree rotations
            k = int(torch.randint(0,4,(1,)).item())
            if k != 0:
                mri = torch.rot90(mri, k, dims=[1,2])
                ct  = torch.rot90(ct,  k, dims=[1,2])

        return mri, ct

# -------------------------
# Paths and loaders
# -------------------------
#split_json = os.path.join(cfg.data_root, "patient_split.json")
#mri_dir = os.path.join(cfg.data_root, "MRI_slices")
#ct_dir  = os.path.join(cfg.data_root, "CT_slices")

split_json = "/kaggle/input/synth2023/processed/patient_split.json"
mri_dir = "/kaggle/input/synth2023/processed/MRI_slices"
ct_dir  = "/kaggle/input/synth2023/processed/CT_slices"

train_dataset = MRICtDataset("train", split_json, mri_dir, ct_dir, cfg.image_size, augment=True)
val_dataset   = MRICtDataset("val", split_json, mri_dir, ct_dir, cfg.image_size, augment=False)

train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True,
                          num_workers=4, pin_memory=True, prefetch_factor=2)
val_loader   = DataLoader(val_dataset, batch_size=min(8, cfg.batch_size), shuffle=True,
                          num_workers=2, pin_memory=True)

print(f"Train:{len(train_dataset)} Val:{len(val_dataset)} ")

# -------------------------
# Models (InstanceNorm instead of BatchNorm)
# Also discriminator returns intermediate feature maps for feature matching
# -------------------------
class TurboUNetGenerator(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, features=64):
        super().__init__()
        def conv_block(in_ch, out_ch, kernel_size=3, stride=1, padding=1, use_in=True):
            layers = [nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding, bias=False)]
            if use_in:
                layers.append(nn.InstanceNorm2d(out_ch, affine=True))
            else:
                layers.append(nn.InstanceNorm2d(out_ch, affine=True))
            layers.append(nn.ReLU(inplace=True))
            return nn.Sequential(*layers)

        def res_block(ch):
            return nn.Sequential(
                conv_block(ch, ch),
                conv_block(ch, ch)
            )

        self.down1 = nn.Sequential(conv_block(in_channels, features, use_in=True), res_block(features))
        self.down2 = nn.Sequential(nn.Conv2d(features, features*2, 4, 2, 1, bias=False),
                                   nn.InstanceNorm2d(features*2, affine=True),
                                   nn.ReLU(inplace=True),
                                   res_block(features*2))
        self.down3 = nn.Sequential(nn.Conv2d(features*2, features*4, 4, 2, 1, bias=False),
                                   nn.InstanceNorm2d(features*4, affine=True),
                                   nn.ReLU(inplace=True),
                                   res_block(features*4))
        self.down4 = nn.Sequential(nn.Conv2d(features*4, features*8, 4, 2, 1, bias=False),
                                   nn.InstanceNorm2d(features*8, affine=True),
                                   nn.ReLU(inplace=True),
                                   res_block(features*8))
        self.down5 = nn.Sequential(nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
                                   nn.InstanceNorm2d(features*8, affine=True),
                                   nn.ReLU(inplace=True),
                                   res_block(features*8))

        self.bottleneck = nn.Sequential(nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
                                        nn.ReLU(inplace=True),
                                        res_block(features*8))

        def up_block(in_ch, out_ch, apply_dropout=False):
            layers = [nn.ConvTranspose2d(in_ch, out_ch, 4, 2, 1, bias=False),
                      nn.InstanceNorm2d(out_ch, affine=True),
                      nn.ReLU(inplace=True)]
            if apply_dropout:
                layers.append(nn.Dropout(0.5))
            return nn.Sequential(*layers)

        self.up1 = up_block(features*8, features*8, apply_dropout=True)
        self.up2 = up_block(features*16, features*8, apply_dropout=True)
        self.up3 = up_block(features*16, features*4)
        self.up4 = up_block(features*8, features*2)
        self.up5 = up_block(features*4, features)
        self.final = nn.Sequential(nn.ConvTranspose2d(features*2, out_channels, 4, 2, 1), nn.Tanh())

    def forward(self, x):
        def crop_to_match(a, b):
            _,_,H,W = b.shape
            return a[:,:,:H,:W]

        d1 = self.down1(x)   # (B, f, H, W)
        d2 = self.down2(d1)  # (B, 2f, H/2, W/2)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        bottleneck = self.bottleneck(d5)

        u1 = self.up1(bottleneck)
        u2 = self.up2(torch.cat([crop_to_match(u1,d5), d5], dim=1))
        u3 = self.up3(torch.cat([crop_to_match(u2,d4), d4], dim=1))
        u4 = self.up4(torch.cat([crop_to_match(u3,d3), d3], dim=1))
        u5 = self.up5(torch.cat([crop_to_match(u4,d2), d2], dim=1))
        out = self.final(torch.cat([crop_to_match(u5,d1), d1], dim=1))
        return crop_to_match(out, x)

# Discriminator returns list of intermediate features + final logits
class PatchDiscriminator(nn.Module):
    def __init__(self, in_channels=2, features=64):
        super().__init__()
        def block(in_ch, out_ch, stride=2, normalize=True):
            layers = [nn.Conv2d(in_ch, out_ch, 4, stride, 1, bias=False)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_ch, affine=True))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return nn.Sequential(*layers)

        self.b1 = block(in_channels, features, normalize=False)   # no norm on first
        self.b2 = block(features, features*2)
        self.b3 = block(features*2, features*4)
        self.b4 = block(features*4, features*8, stride=1)
        self.final = nn.Conv2d(features*8, 1, 4, 1, 1)

    def forward(self, x, y):
        # concat and compute sequentially, collect features
        inp = torch.cat([x, y], dim=1)
        f1 = self.b1(inp)
        f2 = self.b2(f1)
        f3 = self.b3(f2)
        f4 = self.b4(f3)
        out = self.final(f4)
        # return features list and output
        feats = [f1, f2, f3, f4]
        return feats, out

# -------------------------
# Losses, models, optimizers, schedulers, scaler
# -------------------------
netG = TurboUNetGenerator().to(device)
netD = PatchDiscriminator().to(device)

criterion_GAN = nn.BCEWithLogitsLoss()
criterion_L1 = nn.L1Loss()

optimizer_G = optim.Adam(netG.parameters(), lr=cfg.lr_G, betas=(cfg.beta1, 0.999))
optimizer_D = optim.Adam(netD.parameters(), lr=cfg.lr_D, betas=(cfg.beta1, 0.999))

scheduler_G = optim.lr_scheduler.ReduceLROnPlateau(optimizer_G, mode="min", factor=0.5, patience=2, min_lr=1e-6)
scheduler_D = optim.lr_scheduler.ReduceLROnPlateau(optimizer_D, mode="min", factor=0.5, patience=2, min_lr=1e-6)

scaler = GradScaler()  # new API: no args

# resume if checkpoint exists
start_epoch = 0
resume_path = os.path.join(cfg.save_dir, "latest_checkpoint.pth")
best_ssim = -1.0
best_mae  = float("inf")
if os.path.exists(resume_path):
    ck = torch.load(resume_path, map_location=device)
    netG.load_state_dict(ck["netG"])
    netD.load_state_dict(ck["netD"])
    optimizer_G.load_state_dict(ck["optG"])
    optimizer_D.load_state_dict(ck["optD"])
    if "scaler" in ck:
        scaler.load_state_dict(ck["scaler"])
    best_ssim = ck.get("best_ssim", best_ssim)
    best_mae  = ck.get("best_mae", best_mae)
    start_epoch = ck.get("epoch", 0) + 1
    print(f"Resumed from epoch {start_epoch}")

# -------------------------
# Evaluation (vectorized)
# -------------------------
def evaluate_model(netG, loader, max_batches=None):
    netG.eval()
    mae_vals, mse_vals, psnr_vals, ssim_vals = [], [], [], []
    with torch.no_grad():
        for i, (mri, ct) in enumerate(loader):
            mri, ct = mri.to(device), ct.to(device)
            fake_ct = netG(mri)
            ct_np   = ct.squeeze(1).cpu().numpy()
            fake_np = fake_ct.squeeze(1).cpu().numpy()
            mae_vals.append(np.mean(np.abs(ct_np - fake_np), axis=(1,2)))
            mse_vals.append(np.mean((ct_np - fake_np)**2, axis=(1,2)))
            psnr_vals.append(np.array([peak_signal_noise_ratio(gt, pred, data_range=2.0)
                                       for gt, pred in zip(ct_np, fake_np)]))
            ssim_vals.append(np.array([structural_similarity(gt, pred, data_range=2.0)
                                       for gt, pred in zip(ct_np, fake_np)]))
            if max_batches and (i+1 >= max_batches): break
    mae = float(np.concatenate(mae_vals).mean()) if len(mae_vals)>0 else float('nan')
    mse = float(np.concatenate(mse_vals).mean()) if len(mse_vals)>0 else float('nan')
    psnr = float(np.concatenate(psnr_vals).mean()) if len(psnr_vals)>0 else float('nan')
    ssim = float(np.concatenate(ssim_vals).mean()) if len(ssim_vals)>0 else float('nan')
    return mae, mse, psnr, ssim

# -------------------------
# Save sample grid utility
# -------------------------
def save_sample_grid(netG, loader, path, n=4):
    netG.eval()
    imgs = []
    saved = 0
    with torch.no_grad():
        for mri, ct in loader:
            mri, ct = mri.to(device), ct.to(device)
            fake = netG(mri)
            B = mri.shape[0]
            for b in range(B):
                imgs.extend([mri[b], fake[b], ct[b]])
                saved += 1
                if saved >= n: break
            if saved >= n: break
    if len(imgs)==0: return
    grid = torch.stack(imgs, dim=0)
    vutils.save_image(grid, path, nrow=3, normalize=True)

# -------------------------
# Training loop
# -------------------------
val_history = []
G_losses, D_losses = [], []

for epoch in range(start_epoch, cfg.num_epochs):
    netG.train(); netD.train()
    epoch_loss_G = 0.0
    epoch_loss_D = 0.0
    batches = 0

    # compute current lambda_L1 schedule: decay from start -> 10
    lambda_L1 = max(10.0, cfg.lambda_L1_start * (1.0 - epoch / float(cfg.num_epochs)))
    for mri, ct in tqdm(train_loader, leave=False, desc=f"Epoch {epoch+1}/{cfg.num_epochs}"):
        mri, ct = mri.to(device, non_blocking=True), ct.to(device, non_blocking=True)

        # add small gaussian noise to inputs for D
        if cfg.noise_std > 0:
            mri_noisy = mri + cfg.noise_std * torch.randn_like(mri)
            ct_noisy  = ct  + cfg.noise_std * torch.randn_like(ct)
        else:
            mri_noisy, ct_noisy = mri, ct

        # ----------------
        # Generator update (1 step)
        # ----------------
        optimizer_G.zero_grad()
        with autocast("cuda"):
            fake_ct = netG(mri)  # (B,1,H,W)
            feats_fake, pred_fake = netD(mri_noisy, fake_ct)  # feats_fake: list, pred_fake: logits
            # GAN loss (label smoothing)
            valid = torch.full_like(pred_fake, cfg.label_real, device=device)
            loss_GAN = criterion_GAN(pred_fake, valid)
            # L1
            # Convert back to HU before computing MAE
            fake_ct_hu = denormalize(fake_ct)
            ct_hu      = denormalize(ct)
            loss_L1    = criterion_L1(fake_ct_hu, ct_hu) * lambda_L1

            # Feature-matching: get features for real (no grad) and fake (with grad)
            with torch.no_grad():
                feats_real, _ = netD(mri_noisy, ct_noisy)
            fm_loss = 0.0
            for fr, ff in zip(feats_real, feats_fake):
                fm_loss += torch.mean(torch.abs(fr.detach() - ff))
            loss_FM = cfg.lambda_fm * fm_loss
            loss_G = loss_GAN + loss_L1 + loss_FM

        scaler.scale(loss_G).backward()
        scaler.step(optimizer_G)
        scaler.update()

        # ----------------
        # Discriminator update (1 step)
        # ----------------
        optimizer_D.zero_grad()
        with autocast("cuda"):
            feats_real, pred_real = netD(mri_noisy, ct_noisy)
            real_label = torch.full_like(pred_real, cfg.label_real, device=device)
            loss_real = criterion_GAN(pred_real, real_label)

            feats_fake_det, pred_fake_det = netD(mri_noisy, fake_ct.detach())
            fake_label = torch.full_like(pred_fake_det, cfg.label_fake, device=device)
            loss_fake = criterion_GAN(pred_fake_det, fake_label)

            loss_D = 0.5 * (loss_real + loss_fake)

        scaler.scale(loss_D).backward()
        scaler.step(optimizer_D)
        scaler.update()

        # optional: second G step for early warmup could be added here if desired
        # (omitted for simplicity; you can enable if you saw D dominate again)

        epoch_loss_G += loss_G.item()
        epoch_loss_D += loss_D.item()
        batches += 1

    avg_g = epoch_loss_G / max(1, batches)
    avg_d = epoch_loss_D / max(1, batches)
    G_losses.append(avg_g); D_losses.append(avg_d)

    # Validation (only every N epochs)
    if (epoch + 1) % cfg.validate_every == 0:
        mae_val, mse_val, psnr_val, ssim_val = evaluate_model(netG, val_loader, max_batches=cfg.max_val_batches)
        lr_g = optimizer_G.param_groups[0]['lr']; lr_d = optimizer_D.param_groups[0]['lr']
        print(f"Epoch {epoch+1} Val -> MAE: {mae_val:.4f}, MSE: {mse_val:.4f}, PSNR: {psnr_val:.3f}, SSIM: {ssim_val:.4f}, LR_G: {lr_g:.6f}, LR_D: {lr_d:.6f}")

        # Save sample visuals
        sample_path = os.path.join(cfg.save_dir, f"samples_epoch{epoch+1}.png")
        save_sample_grid(netG, val_loader, sample_path, n=cfg.sample_grid_n)

        # Save best models based on SSIM & MAE
        if ssim_val > best_ssim:
            best_ssim = ssim_val
            torch.save(netG.state_dict(), os.path.join(cfg.save_dir, f"best_ssim_epoch{epoch+1}.pth"))
            print("Saved new best SSIM:", best_ssim)
        if mae_val < best_mae:
            best_mae = mae_val
            torch.save(netG.state_dict(), os.path.join(cfg.save_dir, f"best_mae_epoch{epoch+1}.pth"))
            print("Saved new best MAE:", best_mae)

        # Step LR schedulers on MAE (lower is better)
        try:
            scheduler_G.step(mae_val)
            scheduler_D.step(mae_val)
        except Exception as e:
            print("Scheduler step failed:", e)

        # Save checkpoint
        torch.save({
            "epoch": epoch,
            "netG": netG.state_dict(),
            "netD": netD.state_dict(),
            "optG": optimizer_G.state_dict(),
            "optD": optimizer_D.state_dict(),
            "scaler": scaler.state_dict(),
            "best_ssim": best_ssim,
            "best_mae": best_mae
        }, resume_path)

        # record val history row
        val_history.append({
            "epoch": epoch+1,
            "Loss_G": avg_g,
            "Loss_D": avg_d,
            "MAE": mae_val,
            "MSE": mse_val,
            "PSNR": psnr_val,
            "SSIM": ssim_val,
            "LR_G": lr_g,
            "LR_D": lr_d
        })
    else:
        val_history.append({
            "epoch": epoch+1,
            "Loss_G": avg_g,
            "Loss_D": avg_d,
            "MAE": None,
            "MSE": None,
            "PSNR": None,
            "SSIM": None,
            "LR_G": optimizer_G.param_groups[0]['lr'],
            "LR_D": optimizer_D.param_groups[0]['lr']
        })

# Final evaluation
if len(val_loader)>0:
    mae_final, mse_final, psnr_final, ssim_final = evaluate_model(netG, val_loader, max_batches=None)
    print(f"Final Validation -> MAE: {mae_final:.4f}, MSE: {mse_final:.4f}, PSNR: {psnr_final:.3f}, SSIM: {ssim_final:.4f}")

# Save logs CSV
df = pd.DataFrame(val_history)
df.to_csv(os.path.join(cfg.save_dir, "training_log.csv"), index=False)
# Save loss curves
plt.figure(); plt.plot(G_losses, label="G"); plt.plot(D_losses, label="D"); plt.legend(); plt.savefig(os.path.join(cfg.save_dir, "loss_curve.png")); plt.close()

print("Done. Best SSIM:", best_ssim, "Best MAE:", best_mae)


Using device: cuda
Preloading 3780 slices into memory for 'train'...


100%|██████████| 3780/3780 [00:42<00:00, 88.14it/s]


Preloading 540 slices into memory for 'val'...


100%|██████████| 540/540 [00:06<00:00, 87.82it/s]


Train:3780 Val:540 


...

Epoch 60 Val -> MAE: 0.0470, MSE: 0.0183, PSNR: 23.825, SSIM: 0.8167, LR_G: 0.000200, LR_D: 0.000100


Epoch 65 Val -> MAE: 0.0462, MSE: 0.0182, PSNR: 23.820, SSIM: 0.8181, LR_G: 0.000200, LR_D: 0.000100


Epoch 70 Val -> MAE: 0.0444, MSE: 0.0176, PSNR: 24.057, SSIM: 0.8286, LR_G: 0.000100, LR_D: 0.000050
Saved new best SSIM: 0.82860586698846
Saved new best MAE: 0.04437018930912018


Epoch 75 Val -> MAE: 0.0439, MSE: 0.0173, PSNR: 24.089, SSIM: 0.8297, LR_G: 0.000100, LR_D: 0.000050
Saved new best SSIM: 0.8297009755553896
Saved new best MAE: 0.04392577335238457


Epoch 80 Val -> MAE: 0.0442, MSE: 0.0171, PSNR: 24.111, SSIM: 0.8244, LR_G: 0.000100, LR_D: 0.000050
Final Validation -> MAE: 0.0445, MSE: 0.0173, PSNR: 24.082, SSIM: 0.8237
Done. Best SSIM: 0.8297009755553896 Best MAE: 0.04392577335238457


In [3]:
!cp -r /kaggle/working/Model_InstNorm_FM /kaggle/working/output/
